# F05. Generate LULC Map

## F05.00. Initialize Earth Engine

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import ee
import geemap

In [19]:
ee.Authenticate()
ee.Initialize()

## F05.01. Image Acquisition & Composite Generation

### F05.01.A. Set up area and time period

In [20]:
# Input AOI file
admin = ee.FeatureCollection("FAO/GAUL/2015/level1")
sumsel = admin.filter(ee.Filter.eq('ADM0_NAME', 'Indonesia')).filter(
    ee.Filter.eq('ADM1_NAME', 'Sumatera Selatan')
)

# Please add script to upload spatial file

# Input time period
start_date = '2023-01-01'
end_date = '2023-12-31'
year = 2023

In [21]:
# Define map var
Map = geemap.Map() 
Map.addLayer(sumsel, 
            {'color': 'red', 'fillColor': '00000000'}, 
            'Admin Boundaries - Sumatera Selatan')
# Use bounds() to reduce geometry complexity
aoi = sumsel.geometry()

# Visualize AOI
Map.centerObject(aoi, 8)
Map

Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

### F05.01.B. Data collection

In [22]:
# Input image 
collection = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
              .filterBounds(aoi)
              .filterDate(start_date, end_date)
              .filter(ee.Filter.lt('CLOUD_COVER', 20))  # Input value for cloud cover filter
              .limit(50))  # Limit number of images

print('Number of images:', collection.size().getInfo())

Number of images: 27


### F05.01.C. Cloud masking (simplified)

In [23]:
def mask_l9sr(image):
    """Apply cloud masking and scaling to Landsat 9 image"""
    cloud_shadow_bit_mask = (1 << 3)
    clouds_bit_mask = (1 << 5)
    qa = image.select('QA_PIXEL')
    mask = (qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0)
            .And(qa.bitwiseAnd(clouds_bit_mask).eq(0)))
    
    # Apply scaling and masking in one step
    optical_bands = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    scaled_image = image.addBands(optical_bands, None, True)
    
    return (scaled_image.updateMask(mask)
            .copyProperties(image, ['system:time_start']))

### F05.01.D. Generate Composite

In [24]:
# Input bands selection to generate composite
composite = (collection
             .map(mask_l9sr)
             .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'])
             .median()
             .clip(aoi))

Map = geemap.Map()
Map.centerObject(aoi, 8)

Map.addLayer(composite, 
            {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3}, 
            'Landsat 9 Composite (RGB)')

# Display composite
Map

Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

## F05.02. Generate and Select Covariates

### F05.02.A. NDVI and NDWI

In [25]:
# Input bands for generate NDVI and NDWI
ndvi = composite.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
ndwi = composite.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')
evi = composite.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': composite.select('SR_B5'),
        'RED': composite.select('SR_B4'),
        'BLUE': composite.select('SR_B2')
    }).rename('EVI')

composite_with_indices = composite.addBands([ndvi, ndwi, evi])

# Add NDVI layer
ndvi_palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9641']

Map = geemap.Map()
Map.centerObject(aoi, 8)

Map.addLayer(ndvi.clip(aoi), 
            {'min': -1, 'max': 1, 'palette': ndvi_palette}, 
            'NDVI')

# Add NDWI layer
ndwi_palette = ['#8B4513', '#DAA520', '#FFFF00', '#ADFF2F', '#00FF00', '#00FFFF', '#0000FF', '#000080']
Map.addLayer(ndwi.clip(aoi), 
            {'min': -1, 'max': 1, 'palette': ndwi_palette}, 
            'NDWI')
# Display map
Map

Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

## F05.03. Define LULC Categories & Reference Data Preparation

### F05.03.A. Prepare training data

In [26]:
# Input point shapefile
training_points = ee.FeatureCollection('projects/ee-rg2icraf/assets/Sumsel_GT_Restore') #shapefile contained 17 LULC classes

# Check training data size
print('Total training points:', training_points.size().getInfo())

# Stratified sampling to ensure balanced classes
with_random = training_points.randomColumn('random', 42)
training = with_random.filter(ee.Filter.lt('random', 0.7))
validation = with_random.filter(ee.Filter.gte('random', 0.3))  # Fixed: should be 0.7

print('Training points:', training.size().getInfo())
print('Validation points:', validation.size().getInfo())

# Add training and validation points separately
Map = geemap.Map()
Map.centerObject(aoi, 8)

Map.addLayer(training, 
            {'color': 'blue'}, 
            'Training Points')

Map.addLayer(validation, 
            {'color': 'orange'}, 
            'Validation Points')

# Display points data
Map

Total training points: 39003
Training points: 27182
Validation points: 27364


Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

## F05.04. Feature Extraction

### F05.04.A. Optimized sampling

In [27]:
# Input selected bands
bands = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'NDVI', 'NDWI', 'EVI']

# Sample with smaller scale for faster processing
training_samples = composite_with_indices.select(bands).sampleRegions(
    collection=training,
    properties=['kelas'],
    scale=60,  # Increased from 30 to reduce processing
    tileScale=4,  # Add tileScale for memory optimization
    geometries=True
)

validation_samples = composite_with_indices.select(bands).sampleRegions(
    collection=validation,
    properties=['kelas'],
    scale=60,
    tileScale=4,
    geometries=True
)

## F05.05. Model Training & Classification

### F05.05.A. Model training using RandomForest

In [28]:
classifier = ee.Classifier.smileRandomForest(
    # Input hyper-parameter
    numberOfTrees=100,  # Reduced from 200
    variablesPerSplit=3,
    minLeafPopulation=2,  # Increased from 1
    bagFraction=0.7,  # Increased from 0.5
    seed=42
)

trained = classifier.train(
    features=training_samples,
    classProperty='kelas',
    inputProperties=bands
)

### F05.05.B. Classification

In [29]:
# Classify with tileScale for memory optimization
classified = (composite_with_indices.select(bands)
              .classify(trained)
              .set('system:time_start', ee.Date(start_date).millis()))

### F05.05.C. Validation

In [30]:
validated = validation_samples.classify(trained)
confusion_matrix = validated.errorMatrix('kelas', 'classification')

print('=== ACCURACY RESULTS ===')
print('Overall Accuracy:', confusion_matrix.accuracy().getInfo())
print('Kappa Coefficient:', confusion_matrix.kappa().getInfo())


=== ACCURACY RESULTS ===
Overall Accuracy: 0.7961904761904762
Kappa Coefficient: 0.7669751927941418


### F05.05.D. Visualisation

In [31]:
# Define land cover color palette and class names
land_cover_palette = [
    '#006400',  # Undisturbed dry-land forest
    '#228B22',  # Logged-over dry-land forest
    '#4169E1',  # Undisturbed mangrove
    '#87CEEB',  # Logged-over mangrove
    '#2E8B57',  # Undisturbed swamp forest
    '#8FBC8F',  # Logged-over swamp forest
    '#9ACD32',  # Agroforestry
    '#32CD32',  # Plantation forest
    '#8B4513',  # Rubber monoculture
    '#FF8C00',  # Oil palm monoculture
    '#DAA520',  # Other monoculture
    '#ADFF2F',  # Grass/savanna
    '#90EE90',  # Shrub
    '#FFFF00',  # Cropland
    '#FF0000',  # Settlement
    '#D2B48C',  # Cleared land
    '#0000FF'   # Waterbody
]

land_cover_names = [
    'Undisturbed dry-land forest',
    'Logged-over dry-land forest',
    'Undisturbed mangrove',
    'Logged-over mangrove',
    'Undisturbed swamp forest',
    'Logged-over swamp forest',
    'Agroforestry',
    'Plantation forest',
    'Rubber monoculture',
    'Oil palm monoculture',
    'Other monoculture',
    'Grass/savanna',
    'Shrub',
    'Cropland',
    'Settlement',
    'Cleared land',
    'Waterbody'
]

# Create interactive map
Map = geemap.Map()
Map.centerObject(aoi, 8)

# Add RGB composite layer
Map.addLayer(
    composite,
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    'RGB Composite'
)

# Add classified land cover layer
Map.addLayer(
    classified.clip(aoi),
    {'min': 1, 'max': 17, 'palette': land_cover_palette},
    'Land Cover Classification'
)

# Create legend 
legend_dict = dict(zip(land_cover_names, land_cover_palette))
Map.add_legend(
    title="Land Cover Classes",
    legend_dict=legend_dict,
    draggable=True
)

# Add layer control and display
Map.addLayerControl()
Map

Map(center=[-3.2210694545062024, 104.16355582426586], controls=(WidgetControl(options=['position', 'transparen…

### F05.05.E. Export result

In [32]:
def export_to_drive():
    """Export classification results to Google Drive"""
    
    # Export land cover classification
    task1 = ee.batch.Export.image.toDrive(
        image=classified.clip(aoi),
        description=f'LandCover_17Classes_Optimized_{year}',
        folder='Epsitem_Sumsel',
        region=aoi,
        scale=30,
        maxPixels=1e9,
        crs='EPSG:4326',
        fileFormat='GeoTIFF',
        formatOptions={'cloudOptimized': True}
    )